In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/training-data/training/training12.parquet
/kaggle/input/training-data/training/training3.parquet
/kaggle/input/training-data/training/training1.parquet
/kaggle/input/training-data/training/training9.parquet
/kaggle/input/training-data/training/training4.parquet
/kaggle/input/training-data/training/training5.parquet
/kaggle/input/training-data/training/training16.parquet
/kaggle/input/training-data/training/training11.parquet
/kaggle/input/training-data/training/training14.parquet
/kaggle/input/training-data/training/training8.parquet
/kaggle/input/training-data/training/training15.parquet
/kaggle/input/training-data/training/training2.parquet
/kaggle/input/training-data/training/training10.parquet
/kaggle/input/training-data/training/training6.parquet
/kaggle/input/training-data/training/training7.parquet
/kaggle/input/training-data/training/training0.parquet
/kaggle/input/training-data/training/training17.parquet
/kaggle/input/training-data/training/training13.parquet
/k

Notebook idea about this model is from [here](https://www.kaggle.com/code/cdeotte/candidate-rerank-model-lb-0-575)

In [8]:
VER = 5

import pandas as pd, numpy as np
from tqdm.notebook import tqdm
import os, sys, pickle, glob, gc
from collections import Counter
import itertools
# print('We will use RAPIDS version',cudf.__version__)

In [9]:
#Rapids cudf creates co-visitation matrixes 30 times faster than pandas cpu
# !python--version
# !python

/bin/bash: python--version: command not found
Python 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> 
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
KeyboardInterrupt
>>> 

In [12]:
def read_file(f):
    df = pd.read_parquet(f)
    df.ts = (df.ts/1000).astype('int32')
    df['type'] = df['type'].map(lambda i:type_labels[i])
    return df
type_labels = ['clicks','carts','orders']

In [13]:
%%time
data=read_file('/kaggle/input/training-data/training/training0.parquet')
data

CPU times: user 8.31 s, sys: 4.9 s, total: 13.2 s
Wall time: 12.6 s


,session,aid,ts,type
0,0,1517085,1659304800,clicks
1,0,1563459,1659304904,clicks
2,0,1309446,1659367439,clicks
3,0,16246,1659367719,clicks
4,0,1781822,1659367871,clicks
...,...,...,...,...
31492788,749999,118034,1659367111,clicks
31492789,749999,1659890,1659367141,clicks
31492790,749999,1659890,1659367238,carts
31492791,749999,118034,1659367396,clicks


In [14]:
%%time
#Let's create the first co-visitation matrix 
#Carts Orders" Co-visitation Matrix 
#for each type we give a weight 
#here it gives for clicks 1 , carts 6 and orders 3
weights = {0:1,1:6,2:3}
#it says to compute in parts for memory management
#it only uses the tail of each session meaning only the latest data
#session ascending order and timestamp descending order --->firstly the latest products
data = data.sort_values(['session','ts'],ascending = [True,False])
data = data.reset_index(drop = True)
data['n'] = data.groupby('session').cumcount()
data.head(5)

CPU times: user 24 s, sys: 6.1 s, total: 30.1 s
Wall time: 30.2 s


,session,aid,ts,type,n
0,0,161938,1661684983,clicks,0
1,0,1740927,1661684942,clicks,1
2,0,1228848,1661684528,clicks,2
3,0,938007,1661684355,clicks,3
4,0,843110,1661684298,clicks,4


In [15]:
data[data['session']==1]

,session,aid,ts,type,n
276,1,497868,1661714854,clicks,0
277,1,376932,1661705869,clicks,1
278,1,1628317,1661686382,clicks,2
279,1,207905,1661686200,clicks,3
280,1,1464360,1661447432,clicks,4
281,1,1842593,1660857067,clicks,5
282,1,1734061,1660431511,clicks,6
283,1,959544,1659991199,clicks,7
284,1,105393,1659991168,carts,8
285,1,105393,1659991156,clicks,9


In [16]:
%%time
#from these keep only the tail meaning drop if all the n<30
#this is very useful for train test split 
print('Original shape is:',data.shape)
data2  = data.loc[data.n<30].drop('n',axis = 1)
data2.head(5)

Original shape is: (31492793, 5)
CPU times: user 1.01 s, sys: 421 ms, total: 1.43 s
Wall time: 1.37 s


,session,aid,ts,type
0,0,161938,1661684983,clicks
1,0,1740927,1661684942,clicks
2,0,1228848,1661684528,clicks
3,0,938007,1661684355,clicks
4,0,843110,1661684298,clicks


In [17]:
data2.shape

(12380248, 4)

In [18]:
%%time
#because this cell had trouble running that is why it is done in chunks with memory management 
#we are going to continue using only a sample of this.
sessions = data2['session'].unique()
print('Sessions are:',sessions)
import random
sample = random.sample(sessions.tolist(),len(sessions)//10)
print('Some sample sessions are: \n',sample[0:5])
data_sample = data2.loc[data2.session.isin(sample)]
print(data_sample.head(5))
# data_sample.shape
#create pairs by merging 
data_sample = pd.merge(data_sample,data_sample, on = 'session')
data_sample.head(5)

Sessions are: [     0      1      2 ... 749997 749998 749999]
Some sample sessions are: 
 [160543, 29939, 643274, 56072, 727488]
     session      aid          ts    type
341        3   925352  1661109666   carts
342        3   925352  1661109664  clicks
343        3   925352  1661109645  clicks
344        3  1431783  1661109621  clicks
345        3  1261998  1661109604  clicks
CPU times: user 3.47 s, sys: 2.05 s, total: 5.52 s
Wall time: 5.52 s


,session,aid_x,ts_x,type_x,aid_y,ts_y,type_y
0,3,925352,1661109666,carts,925352,1661109666,carts
1,3,925352,1661109666,carts,925352,1661109664,clicks
2,3,925352,1661109666,carts,925352,1661109645,clicks
3,3,925352,1661109666,carts,1431783,1661109621,clicks
4,3,925352,1661109666,carts,1261998,1661109604,clicks


In [19]:
%%time
#after the merge keep only the pairs that we need based on our needs 
#keep the ones that are chosen the same day (<1) and are different
data_sample  = data_sample.loc[(abs(data_sample.ts_x-data_sample.ts_y)<24*60*60) & (data_sample.aid_x!=data_sample.aid_y)][['session','aid_x','aid_y','type_y']].drop_duplicates()

data_sample.head(5)

CPU times: user 7.08 s, sys: 3.68 s, total: 10.8 s
Wall time: 10.5 s


,session,aid_x,aid_y,type_y
3,3,925352,1431783,clicks
4,3,925352,1261998,clicks
5,3,925352,1575102,clicks
7,3,925352,1261998,carts
9,3,925352,1251514,clicks


In [20]:
#assign weights to each type --->Here more weight to carts and orders from dict called weights in the beggining
weights = {'clicks':1,'carts':6,'orders':3}
data_sample['weight'] = data_sample.type_y.map(weights).astype('float32')


In [21]:
data_sample.head(5)

,session,aid_x,aid_y,type_y,weight
3,3,925352,1431783,clicks,1.0
4,3,925352,1261998,clicks,1.0
5,3,925352,1575102,clicks,1.0
7,3,925352,1261998,carts,6.0
9,3,925352,1251514,clicks,1.0


In [22]:
#keep only the pairs and session
data_sample = data_sample[['aid_x','aid_y','weight']]
data_sample.head(3)

,aid_x,aid_y,weight
3,925352,1431783,1.0
4,925352,1261998,1.0
5,925352,1575102,1.0


In [23]:
#add all the weights together to see 
#however if later there is a need to add more parquet files perhaps there are more to add here
df = data_sample.groupby(['aid_x','aid_y']).weight.sum()
df.head(5)

aid_x  aid_y 
3      16778     2.0
       64692     1.0
       145227    1.0
       170046    9.0
       179560    1.0
Name: weight, dtype: float32

In [24]:
df = df.reset_index()
df = df.sort_values(['aid_x','weight'],ascending = [True,False])
df

,aid_x,aid_y,weight
3,3,170046,9.0
8,3,347257,8.0
15,3,831065,8.0
17,3,1180285,8.0
23,3,1771163,8.0
...,...,...,...
5876657,1855602,181086,1.0
5876658,1855602,289237,1.0
5876659,1855602,531942,1.0
5876660,1855602,721406,1.0


In [25]:
#after sorting the pairs and their weights
#we need to save the top 15 for each aid_x (from discussion it seems it has been tested also for top 40 or 20 but this has better results)
df['n'] = df.groupby('aid_x')['aid_y'].cumcount()
df = df.loc[df.n<15].drop('n',axis = 1)
df

,aid_x,aid_y,weight
3,3,170046,9.0
8,3,347257,8.0
15,3,831065,8.0
17,3,1180285,8.0
23,3,1771163,8.0
...,...,...,...
5876657,1855602,181086,1.0
5876658,1855602,289237,1.0
5876659,1855602,531942,1.0
5876660,1855602,721406,1.0


In [26]:
#this final product will be saved as a parquet
#SAVE PART TO DISK (convert to pandas first uses less memory) !!!
df.to_parquet('top_15_carts_orders.parquet')

### Second co-visitation matrix Buy-2-Buy 
Same logic as before but create pairs with different conditions.            
In this case, we are only interested in carts and orders.

In [33]:
sessions = data2['session'].unique()
print('Sessions are:',sessions)
import random
sample = random.sample(sessions.tolist(),len(sessions)//10)
print('Some sample sessions are: \n',sample[0:5])
data_sample = data2.loc[data2.session.isin(sample)]
print(data_sample.head(5))

Sessions are: [     0      1      2 ... 749997 749998 749999]
Some sample sessions are: 
 [250205, 557114, 290609, 245110, 641166]
     session      aid          ts    type
276        1   497868  1661714854  clicks
277        1   376932  1661705869  clicks
278        1  1628317  1661686382  clicks
279        1   207905  1661686200  clicks
280        1  1464360  1661447432  clicks


In [34]:
#same as before using now only the sample data for quick implementation
#we are going to try and create tha same way a new matrix 
#keep only the tail-last 30 transactions 
#AND ONLY CARTS AND ORDERS 
data = data_sample.loc[data_sample.type.isin(['carts','orders'])]
data = data.sort_values(['session','ts'],ascending = [True,False])
data['n'] = data.groupby('session')['aid'].cumcount()
data = data.loc[data.n<30].drop('n',axis=1)
data.head(3)

,session,aid,ts,type
284,1,105393,1659991168,carts
287,1,711125,1659991053,carts
290,1,215311,1659990964,carts


In [35]:
#after keeping only the latest data, let's create the pairs 
pairs = pd.merge(data,data,on = 'session')
pairs.head(5)

,session,aid_x,ts_x,type_x,aid_y,ts_y,type_y
0,1,105393,1659991168,carts,105393,1659991168,carts
1,1,105393,1659991168,carts,711125,1659991053,carts
2,1,105393,1659991168,carts,215311,1659990964,carts
3,1,105393,1659991168,carts,854637,1659990941,carts
4,1,105393,1659991168,carts,1491172,1659385945,carts


In [36]:
#from these pairs, keep only the ones that meet the conditions 
#Only carts & orders that have a period of 14 days between them 
#All pairs must be different 
pairs = pairs.loc[((pairs.ts_x-pairs.ts_y)<24*60*60*14) & (pairs.aid_x!=pairs.aid_y)]
pairs.head(3)

,session,aid_x,ts_x,type_x,aid_y,ts_y,type_y
1,1,105393,1659991168,carts,711125,1659991053,carts
2,1,105393,1659991168,carts,215311,1659990964,carts
3,1,105393,1659991168,carts,854637,1659990941,carts


In [37]:
#from these keep some columns and drop the duplicates for pair of products for each session
pairs = pairs[['session','aid_x','aid_y','type_y']].drop_duplicates(['session','aid_x','aid_y'])
pairs


,session,aid_x,aid_y,type_y
1,1,105393,711125,carts
2,1,105393,215311,carts
3,1,105393,854637,carts
4,1,105393,1491172,carts
5,1,105393,910862,carts
...,...,...,...,...
793297,749909,1784418,1466314,carts
793300,749913,864909,642804,orders
793301,749913,642804,864909,carts
793304,749939,544304,96121,carts


In [38]:
#We can even add a weight to this like before --->If we leave the weight to 1, it acts like a Counter()
pairs['weight']=1
pairs.weight = pairs.weight.astype('float32')
#we no longer need the session info 
pairs  = pairs[['aid_x','aid_y','weight']]
pairs

,aid_x,aid_y,weight
1,105393,711125,1.0
2,105393,215311,1.0
3,105393,854637,1.0
4,105393,1491172,1.0
5,105393,910862,1.0
...,...,...,...
793297,1784418,1466314,1.0
793300,864909,642804,1.0
793301,642804,864909,1.0
793304,544304,96121,1.0


In [39]:
pairs = pairs.groupby(['aid_x','aid_y']).weight.sum()
#get for every product their counts
pairs

aid_x    aid_y  
3        580439     1.0
         767034     1.0
         1771163    1.0
22       109382     1.0
         222091     1.0
                   ... 
1855582  1672277    1.0
1855594  664409     1.0
         876493     1.0
         1091155    1.0
         1675526    1.0
Name: weight, Length: 283370, dtype: float32

In [42]:
pairs = pairs.reset_index()
pairs = pairs.sort_values(['aid_x','weight'],ascending = [True,False])
pairs

,aid_x,aid_y,weight
0,3,580439,1.0
1,3,767034,1.0
2,3,1771163,1.0
3,22,109382,1.0
4,22,222091,1.0
...,...,...,...
283365,1855582,1672277,1.0
283366,1855594,664409,1.0
283367,1855594,876493,1.0
283368,1855594,1091155,1.0


In [44]:
#from these save for each the top 15
pairs = pairs.reset_index(drop = True)
pairs


,aid_x,aid_y,weight
0,3,580439,1.0
1,3,767034,1.0
2,3,1771163,1.0
3,22,109382,1.0
4,22,222091,1.0
...,...,...,...
283365,1855582,1672277,1.0
283366,1855594,664409,1.0
283367,1855594,876493,1.0
283368,1855594,1091155,1.0


In [45]:
pairs['n']=pairs.groupby('aid_x').aid_y.cumcount()
pairs = pairs.loc[pairs.n<15].drop('n',axis = 1)
pairs

,aid_x,aid_y,weight
0,3,580439,1.0
1,3,767034,1.0
2,3,1771163,1.0
3,22,109382,1.0
4,22,222091,1.0
...,...,...,...
283365,1855582,1672277,1.0
283366,1855594,664409,1.0
283367,1855594,876493,1.0
283368,1855594,1091155,1.0


In [47]:
pairs.to_parquet('top_15_buy_2_buy_v2.parquet')

### 3rd co-visitation matrix
Clicks---> Time weighted 

In [55]:
#continuing with the data sample 
#again use only the tail 
data = data_sample.sort_values(['session','ts'],ascending = [True,False])
data = data.reset_index(drop = True)
data['n'] = data.groupby('session').cumcount()
data = data.loc[data.n<30].drop('n',axis = 1)
data

,session,aid,ts,type
0,1,497868,1661714854,clicks
1,1,376932,1661705869,clicks
2,1,1628317,1661686382,clicks
3,1,207905,1661686200,clicks
4,1,1464360,1661447432,clicks
...,...,...,...,...
1234942,749959,1392191,1660050366,clicks
1234943,749959,1398020,1660050332,clicks
1234944,749959,1259136,1660050311,clicks
1234945,749959,1783283,1660050280,clicks


In [56]:
#create the pairs 
pairs = pd.merge(data,data,on = 'session')
#keep those that are chosen the same day
pairs = pairs.loc[(abs(pairs.ts_x-pairs.ts_y)<24*60*60) & (pairs.aid_x!=pairs.aid_y)]
pairs

,session,aid_x,ts_x,type_x,aid_y,ts_y,type_y
1,1,497868,1661714854,clicks,376932,1661705869,clicks
2,1,497868,1661714854,clicks,1628317,1661686382,clicks
3,1,497868,1661714854,clicks,207905,1661686200,clicks
30,1,376932,1661705869,clicks,497868,1661714854,clicks
32,1,376932,1661705869,clicks,1628317,1661686382,clicks
...,...,...,...,...,...,...,...
30193780,749959,1783283,1660050280,clicks,14652,1660050417,clicks
30193781,749959,1783283,1660050280,clicks,1281206,1660050400,clicks
30193782,749959,1783283,1660050280,clicks,1392191,1660050366,clicks
30193783,749959,1783283,1660050280,clicks,1398020,1660050332,clicks


In [57]:
#from these keep only session,aid_x,aid_y and ts_x (one timestamp)
pairs = pairs[['session','aid_x','aid_y','ts_x']].drop_duplicates(['session','aid_x','aid_y'])
pairs


,session,aid_x,aid_y,ts_x
1,1,497868,376932,1661714854
2,1,497868,1628317,1661714854
3,1,497868,207905,1661714854
30,1,376932,497868,1661705869
32,1,376932,1628317,1661705869
...,...,...,...,...
30193780,749959,1783283,14652,1660050280
30193781,749959,1783283,1281206,1660050280
30193782,749959,1783283,1392191,1660050280
30193783,749959,1783283,1398020,1660050280


##### !!! Time-weighted :Εξίσωση ευθείας με 2 σημεία 
The value 1659304800 is the minimum timestamp in all the data. And the value 1662328791 is the maximum timestamp in all the data. And the max weight we want is 4 and the min weight we want is 1. Therefore

x2 = 1662328791
x1 = 1659304800
y2 = 4
y1 = 1       
Then we use the "point-slope form" equation of a line explained.       
Τhe "4" is arbitrary. We set the min to 1. Then we try different maxes and see which has the best CV score.

Logic behind number 4 : There are 5 weeks of data to train our models (4 train 1 test). The max value determines how much emphasis we give to recent data. If we make the maximum 6 for example, then pairs of items during the last week of test data (most recent) will have weights between 5 and 6. Then second to last week will have weights 4 to 5. And then 3 to 4, 2 to 3, and 1 to 2. (i.e. the most recent week is approx 3.7x more important than the oldest week in determining the pairs in our co-visitation matrix).

In [58]:
#according to this above let's create a new column for the weights
pairs['weight'] = 1+3*(pairs.ts_x-1659304800)/(1662328791-1659304800)
pairs


,session,aid_x,aid_y,ts_x,weight
1,1,497868,376932,1661714854,3.390934
2,1,497868,1628317,1661714854,3.390934
3,1,497868,207905,1661714854,3.390934
30,1,376932,497868,1661705869,3.382020
32,1,376932,1628317,1661705869,3.382020
...,...,...,...,...,...
30193780,749959,1783283,14652,1660050280,1.739566
30193781,749959,1783283,1281206,1660050280,1.739566
30193782,749959,1783283,1392191,1660050280,1.739566
30193783,749959,1783283,1398020,1660050280,1.739566


In [59]:
#after creating everything, keep only the pairs and weight and count them 
pairs = pairs[['aid_x','aid_y','weight']]
pairs.weight = pairs.weight.astype('float32')
pairs = pairs.groupby(['aid_x','aid_y']).weight.sum()

pairs


/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


aid_x    aid_y  
1        106612     2.194684
         408008     2.355325
         748990     2.194684
         1473899    2.194684
         1630292    2.194684
                      ...   
1855602  1433820    1.199262
         1609874    1.199262
         1667461    1.199262
         1671190    1.199262
         1765079    1.199262
Name: weight, Length: 5845438, dtype: float32

In [60]:
#after creating all the pairs and weights we need to keep only the top 15. 
pairs = pairs.reset_index()
pairs = pairs.sort_values(['aid_x','weight'],ascending = [True,False])
pairs = pairs.reset_index(drop=True)
pairs['n'] = pairs.groupby(['aid_x']).aid_y.cumcount()
pairs

,aid_x,aid_y,weight,n
0,1,1835190,4.550009,0
1,1,408008,2.355325,1
2,1,106612,2.194684,2
3,1,748990,2.194684,3
4,1,1473899,2.194684,4
...,...,...,...,...
5845433,1855602,1433820,1.199262,20
5845434,1855602,1609874,1.199262,21
5845435,1855602,1667461,1.199262,22
5845436,1855602,1671190,1.199262,23


In [61]:
pairs = pairs.loc[pairs.n<15].drop('n',axis=1)
pairs

,aid_x,aid_y,weight
0,1,1835190,4.550009
1,1,408008,2.355325
2,1,106612,2.194684
3,1,748990,2.194684
4,1,1473899,2.194684
...,...,...,...
5845423,1855602,834935,1.199262
5845424,1855602,888826,1.199262
5845425,1855602,999405,1.199262
5845426,1855602,1119153,1.199262


In [62]:
pairs.to_parquet('top_15_clicks_v3.parquet')

### In order to free memory del data_cache!

In [75]:
del pairs
del data,data2
del df
gc.collect()

### Re-Ranking of these methods and their submissions
Can use 3 XGBoost, one for each type clicks,cartsand orders.

In [78]:
test = read_file('/kaggle/input/testing-data/testing/testing_final0.parquet')
test.head(5)

,session,aid,ts,type
0,12899779,59625,1661724000,clicks
1,12899780,1142000,1661724000,clicks
2,12899780,582732,1661724058,clicks
3,12899780,973453,1661724109,clicks
4,12899780,736515,1661724136,clicks


In [84]:
%%time
#function to read the predictions previously and convert them to a dictionary 
    
top_15_carts_orders = pd.read_parquet('/kaggle/working/top_15_carts_orders.parquet')
top_15_carts_orders=top_15_carts_orders.reset_index(drop=True)
top_15_carts_orders=top_15_carts_orders.groupby('aid_x').aid_y.apply(list).to_dict()
# top_15_carts_orders

CPU times: user 6.81 s, sys: 350 ms, total: 7.16 s
Wall time: 6.91 s


In [85]:
#same thing for the other two submissions
top_15_buys = pd.read_parquet('/kaggle/working/top_15_buy_2_buy_v2.parquet')
top_15_buys=top_15_buys.reset_index(drop=True)
top_15_buys=top_15_buys.groupby('aid_x').aid_y.apply(list).to_dict()
top_15_clicks = pd.read_parquet('/kaggle/working/top_15_clicks_v3.parquet')
top_15_clicks=top_15_clicks.reset_index(drop=True)
top_15_clicks=top_15_clicks.groupby('aid_x').aid_y.apply(list).to_dict()

In [93]:
#find the top clicks and orders in the test dataset
top_clicks = test.loc[test['type']=='clicks','aid'].value_counts().index.values[:20]
top_clicks
top_orders = test.loc[test['type']=='orders','aid'].value_counts().index.values[:20]
top_orders

array([1495817,  986164, 1460571, 1043508,  332654,  579690,  923948,
       1457846,  544144,   29735,  585186, 1022566, 1006198,  166037,
       1236775,  892871,  329725,  823143, 1359971,  714524])

In [94]:
#it's only a sample of the data still
print(len(top_15_carts_orders),len(top_15_buys),len(top_15_clicks))

303191 52274 302144


In [ ]:
#for each type of action give a separate weight
type_mult = {0:1,1:6,2:3}

In [100]:
#for clicks 
def suggest_clicks(df):
    #keep the products in a list
    aids = df.aid.tolist()
    types = df.type.tolist()
    unique_aids = list(dict.fromkeys(aids[::-1] ))
    #rerank with weights
    if(len(unique_aids)>=20):
        weights =np.logspace(0.1,1,len(aids),base=2, endpoint=True)-1
    #....it continues from the notebook 

In [101]:
pred_df_clicks = test.sort_values(["session", "ts"]).groupby(["session"]).apply(
    lambda x: suggest_clicks(x)
)

In [ ]:
pred_df_clicks